### This work is made in a benefit from a tutorial:
https://www.kaggle.com/joelowj/a-sentiment-analysis-tutorial/notebook

In [1]:
import pandas as pd
from sklearn.utils import shuffle

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Load all datas
data = pd.read_csv("/home/emremrah/Desktop/DataMin/twitter-sentiment/sample-train-test.csv",
                   encoding='latin-1')
data_train = pd.read_csv("/home/emremrah/Desktop/DataMin/twitter-sentiment/train.csv",
                        encoding='latin-1')
data_test = pd.read_csv("/home/emremrah/Desktop/DataMin/twitter-sentiment/test.csv",
                       encoding='latin-1')

# Shuffle all of datas
data = shuffle(data, random_state=144)
data_train = shuffle(data_train, random_state=144)
data_test = shuffle(data_test, random_state=144)

In [3]:
data = data.drop(columns='ItemID')
data.head()

,Sentiment,SentimentText
82984,1,@AshNair OMGGGG Ash Nair. IM A BIG FAN OF YOU...
98698,1,"@CottonCandyKiss yay, i've added her! She is s..."
64272,1,@BinkleyOnStyle 55+ - No Problem! My battle ax...
6788,1,"#myweakness - my phoneeeee without it, i'd be..."
63001,1,@billyraycyrus http://twitpic.com/6p0vb - gorg...


In [4]:
import re

def preprocess(text):
    try:
        text = re.sub(r'@[A-Za-z0-9_]+', '', text) # remove @ mentions fron tweets
    except:
        text = text
    try:
        text = re.sub('<[^>]*>', ' ', text)    # removes HTML from tweets
    except:
        text = text
    try:
        text = re.sub('(http|https)://[^ ]+ ', '', text)    # removes all the hyperlinks
    except:
        text = text
    try:
        text = re.sub('\s\s+', '', text)    # removes all the extra whitespaces
    except:
        text = text
    try:
        emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P|[^T_T])', text)    #find all emoticons
    except:
        text = text
    try:
        text = re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', '')  # appends emmoticons at the end.
    except:
        text = text
    return text

In [5]:
# Clean tweets from hyperlinks, '@' ect.

def clean_tweets (df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

In [6]:
from nltk.tokenize import TweetTokenizer

# First preprocess the texts (tweets), then tokenize them with nltk TweetTokenizer
tokenizer = TweetTokenizer()
data['SentimentText'] = data['SentimentText'].apply(preprocess)
data = clean_tweets(data, "SentimentText")

data['Tokens'] = data["SentimentText"].apply(tokenizer.tokenize)
data.head()

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


,Sentiment,SentimentText,Tokens
82984,1,omgggg ash nair im a big fan of you you daym h...,"[omgggg, ash, nair, im, a, big, fan, of, you, ..."
98698,1,yay i ve added her she is so damn hearty lt 3 3,"[yay, i, ve, added, her, she, is, so, damn, he..."
64272,1,55 no problem my battle axe is a boomer older...,"[55, no, problem, my, battle, axe, is, a, boom..."
6788,1,myweakness my phoneeeeewithout it i d be d e ...,"[myweakness, my, phoneeeeewithout, it, i, d, b..."
63001,1,gorgeous horse,"[gorgeous, horse]"


In [7]:
all_words = [word for tokens in data['Tokens'] for word in tokens]
vocab = sorted(list(set(all_words)))
print("Total words: {} means total words in all of tweets\n".format(len(all_words)) + 
      "Vocabulary size: {} means number of different words in all tweets".format(len(vocab)))

Total words: 1285766 means total words in all of tweets
Vocabulary size: 66817 means number of different words in all tweets


In [39]:
# CountVectorizer is a bag-of-words representation implemented in scikit-learn
# Tfidf explained briefly in 'O'Reilly Introduction  to Machine Learning'
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

list_corpus = data['SentimentText'].tolist()
list_labels = data['Sentiment'].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.15,
                                                   random_state=44)

vect = CountVectorizer(min_df=2, ngram_range=(1, 3)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_test_vectorized = vect.transform(X_test)

### LogReg with defaults vectors

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [41]:
params_grid = {'C': [0.1, 1]}
lr_grid = GridSearchCV(LogisticRegression(), params_grid, cv=5)
lr_grid.fit(X_train_vectorized, y_train)
print("Best score: {:.4f}".format(lr_grid.best_score_))
print("Best parameters: ", lr_grid.best_params_)

Best score: 0.7679
Best parameters:  {'C': 0.1}


In [42]:
lr_best = LogisticRegression(C=0.1).fit(X_train_vectorized, y_train)
print("Training accuracy: {:.4f}".format(lr_best.score(X_train_vectorized, y_train)))
print("Testing accuracy: {:.4f}".format(lr_best.score(X_test_vectorized, y_test)))

Training accuracy: 0.8500
Testing accuracy: 0.7706


With default vocabulary settings, Logistic Regression made a presicion around %75 on test set.
Now we try to change some parameters, like n-gram range, min-df

With parameters changed, our training score is slightly increased to %76

##### With ngram_range=(1, 5), there was over 350k vocabulary size, so over 350k fetures. With ngram_range=(1, 2), there is around 79k, wich is much less and the precision not even reduced. But the computational cost reduced drastically.

And without cross-validation, with best parameters, our training score is 83% and test score is 76%. We have slightly overfitting here. 

In [43]:
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

predictions = lr_best.predict(X_test_vectorized)
print(confusion_matrix(y_test, predictions, labels=[1, 0]))

[[7175 1322]
 [2119 4383]]


##### As seen, our model predicted most of 'negatives' as 'positives'. Also test scenario seems a bit unbalanced since there is  17k of positives and 13k of negatives. Would be better if there is 15k-15k.

In [ ]:
# Create graph for different parameters

train_acc = []
test_acc = []
best_test=0
train_score=0
best_train=0
for max_depth in range(1, 7):
    for learning_rate in [1]:
        gbm=GradientBoostingClassifier(max_depth=max_depth,
                                       learning_rate=learning_rate,
                                       random_state=44).fit(X_train_counts, y_train)
        train_score= gbm.score(X_train_counts, y_train)
        train_acc.append(gbm.score(X_train_counts, y_train))
        test_acc.append(gbm.score(X_test_counts, y_test))
        if train_score > best_train:
            best_train=train_score
            best_test=gbm.score(X_test_counts, y_test)
            best_parameters = {'max_depth': max_depth, 'learning_rate': learning_rate}

print("Best train score: {:.5f}".format(best_train))
print("Best test score: {:.5f}".format(best_test))
print("Best parameters: {}".format(best_parameters))
plt.plot(range(1, 7), train_acc, label="TRAIN")
plt.plot(range(1, 7), test_acc, label="TEST")
plt.ylabel("Accuracy")
plt.xlabel("iterasyon")
plt.legend()
plt.show()